In [250]:
import numpy as np
import IPython.display as ipd  
import matplotlib.pyplot as plt

fs = 48000

In [251]:
const = lambda y : lambda x : 0*x[0] + y
mult = lambda x : x[0] * x[1]
plus = lambda x : x[0] + x[1]
wav = lambda x : np.piecewise(x[0], [x[0]%1 <= 0.5], [-1,1])
env = lambda x : np.piecewise(x[0], [x[0] < 0, x[0] > x[1]], [0,0,1])

In [299]:
def compute(A,ops):
    
    root = np.where(np.logical_not(np.any(A,axis=1)))[0][0] 
    B = np.stack((np.identity(A.shape[0]),A))
    
    while np.any(B[-1]):
        B = np.vstack((B,(B[1] @ B[-1])[np.newaxis,:]))
    
    paths = np.heaviside(B[:,:,root],0).astype(int)
    order = paths.shape[0] - np.argmax(paths[::-1], axis=0) - 1
    
    values = [None for op in ops]
    
    for n in range(max(order) + 1):
        inds = np.where(order == n)[0]
        for k in inds:
            input_inds = np.where(A[k])[0]
            input_vals = [values[l] for l in input_inds]
            values[k] = ops[k](input_vals)
            
    return values[np.argmax(order)]

In [300]:
R = const(-0.5)
F = const(200)
L = const(1)
x = lambda x : np.arange(0,2,1/fs)

ops = [mult,env,wav,plus,L,mult,R,F,x]

A = np.array(
    [[ 0, 1, 1, 0, 0, 0, 0, 0, 0], \
     [ 0, 0, 0, 1, 1, 0, 0, 0, 0], \
     [ 0, 0, 0, 0, 0, 1, 0, 0, 0], \
     [ 0, 0, 0, 0, 0, 0, 1, 0, 1], \
     [ 0, 0, 0, 0, 0, 0, 0, 0, 1], \
     [ 0, 0, 0, 0, 0, 0, 0, 1, 1], \
     [ 0, 0, 0, 0, 0, 0, 0, 0, 1], \
     [ 0, 0, 0, 0, 0, 0, 0, 0, 1], \
     [ 0, 0, 0, 0, 0, 0, 0, 0, 0]  ] )

In [301]:
test = compute(A,ops)
ipd.display(ipd.Audio(test, rate=fs))  